# Treinamento do Modelo

### Requisitos

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Configuração para ignorar warnings
import warnings
warnings.filterwarnings('ignore')

# Configuração automática para melhor desempenho
AUTO = tf.data.experimental.AUTOTUNE

### 1. Carregamento e Preparação dos Dados

In [2]:
def load_dataset(dataset_path):
    # Carrega todos os caminhos das imagens
    images = glob(f"{dataset_path}/*/*.jpg")
    
    # Padroniza os caminhos
    images = [path.replace('\\', '/') for path in images]
    
    # Cria DataFrame com caminhos e labels
    df = pd.DataFrame({'filepath': images})
    
    df['label'] = df['filepath'].str.split('/', expand=True)[2]  # Assume que a pasta é o label
    
    # Converte labels para valores binários (0 para benigno, 1 para maligno)
    df['label'] = np.where(df['label'] == 'Malignant', 1, 0)
    
    return df

# Carrega o dataset
dataset_path = "../dataset"  # Substitua pelo seu caminho
df = load_dataset(dataset_path)

# Verifica distribuição das classes
print(df['label'].value_counts())

label
0    6289
1    5590
Name: count, dtype: int64


### 2. Divisão do Dataset e Pré-processamento

| Parâmetro | Descrição | Valor | Justificativa
| --- | --- | --- | --- |
| `random_state` | Controla a aleatorização da divisão | 42 | Convenção da comunidade (referência ao "Guia do Mochileiro das Galáxias") para garantir reprodutibilidade | 
| `stratify` | Mantém proporção das classes | `df['label']` | Garante que conjuntos de treino/validação mantenham a mesma distribuição de câncer/não-câncer que o dataset original |
| `num_parallel_calls` | Número de processamentos paralelos | AUTO | Permite que o TensorFlow ajuste automaticamente o paralelismo conforme o hardware disponível | 
| `shuffle` | Embaralha os dados | `True` (treino), `False` (validação) | Evite overfitting durante treino, enquanto validação não precisa ser embaralhada |
| `batch_size` | Tamanho do lote | 32 | Balanceamento entre eficiência de memória e estabilidade do gradiente (valor comum na literatura) |

In [3]:
# Divide em treino e validação
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Função para decodificar e pré-processar imagens
def decode_image(filepath, label=None, image_size=(224, 224)):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, image_size)
    img = tf.cast(img, tf.float32) / 255.0
    
    if label is not None:
        return img, label
    return img

# Cria pipelines de dados para treino e validação
def create_dataset(df, batch_size=32, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label'].values))
    
    dataset = dataset.map(lambda x, y: (decode_image(x, y)), 
                        num_parallel_calls=AUTO)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTO)
    
    return dataset

# Cria datasets
batch_size = 32
train_ds = create_dataset(train_df, batch_size=batch_size, shuffle=True)
val_ds = create_dataset(val_df, batch_size=batch_size)

## 3. Construção do Modelo com Transfer Learning

| Parâmetro | Descrição | Valor | Justificativa
| --- | --- | --- | --- |
| `DenseNet121` | Arquitetura base do modelo | - | Boa relação acurácia-eficiência para imagens médicas dermatológicas |
| `include_top` | Inclui camadas fully-connected | `False` | Permite substituir as camadas finais por novas adequadas ao problema binário |
| `pooling` | Tipo de pooling final | 'avg' |  Global Average Pooling reduz dimensionalidade preservando informações espaciais importantes |
| `input_shape` | Dimensões da imagem | (224, 224, 3) | Tamanho padrão compatível com DenseNet121 |
| `base_model.trainable` | Congela camadas base | `False` | Evita sobre-ajuste inicial e acelera treinamento nas primeiras épocas | 
| `activation` | Camadas personalizadas | 256 (ReLU), 1 (sigmoid) | ReLU para não-linearidade nas hidden layers, sigmoid para saída binária (adequado para classificação câncer/não-câncer) | 
| `BatchNormalization` | Normalização por lote | Após Dense(256) | Estabiliza treinamento e acelera convergência | 
| `learning_rate` | Otimizador | 1e-3 | Taxa de aprendizado padrão para transfer learning |

In [ ]:
def build_model(input_shape=(224, 224, 3)):
    base_model = DenseNet121(
        input_shape=input_shape,
        include_top=False,
        weights=None,
        pooling='avg'
    )
    
    # Congela as camadas do modelo base
    base_model.trainable = False
    
    # Adiciona camadas personalizadas
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    
    return model

# Compila o modelo
model = build_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

## 4. Treinamento do Modelo

In [ ]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_auc',
    mode='max',
    save_best_only=True
)

# Hiperparâmetros
epochs = 20

# Treina o modelo
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint]
)

### 5. Avaliação e Visualização dos Resultados

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

def plot_history(history):
    # === Accuracy ===
    fig, ax = plt.subplots()
    ax.plot(history['accuracy'], label='Acurácia treino', color='#7281EE')
    ax.plot(history['val_accuracy'], label='Acurácia validação', color='#D286EA')

    ax.set_title('Acurácia durante o treino', color='white')
    ax.set_xlabel('Épocas', color='white')
    ax.set_ylabel('Acurácia', color='white')
    ax.tick_params(colors='white')
    ax.legend()
    ax.grid(True, color='white', linestyle='--', linewidth=0.2)
    ax.set_facecolor('#242424')
    fig.patch.set_facecolor('#242424')
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.show()

    # === Loss ===
    fig, ax = plt.subplots()
    ax.plot(history['loss'], label='Perda treino', color='#7281EE')
    ax.plot(history['val_loss'], label='Perda validação', color='#D286EA')

    ax.set_title('Perda durante o treino', color='white')
    ax.set_xlabel('Épocas', color='white')
    ax.set_ylabel('Perda', color='white')
    ax.tick_params(colors='white')
    ax.legend()
    ax.grid(True, color='white', linestyle='--', linewidth=0.2)
    ax.set_facecolor('#242424')
    fig.patch.set_facecolor('#242424')
    for spine in ax.spines.values():
        spine.set_visible(False)
    plt.show()

    # === AUC (Opcional) ===
    if 'auc' in history and 'val_auc' in history:
        fig, ax = plt.subplots()
        ax.plot(history['auc'], label='AUC treino', color='#7281EE')
        ax.plot(history['val_auc'], label='AUC validação', color='#D286EA')

        ax.set_title('AUC durante o treino', color='white')
        ax.set_xlabel('Épocas', color='white')
        ax.set_ylabel('AUC', color='white')
        ax.tick_params(colors='white')
        ax.legend()
        ax.grid(True, color='white', linestyle='--', linewidth=0.2)
        ax.set_facecolor('#242424')
        fig.patch.set_facecolor('#242424')
        for spine in ax.spines.values():
            spine.set_visible(False)
        plt.show()

plot_history(history.history)

### 6. Fine-tuning

| Parâmetro | Descrição | Valor | Justificativa
| --- | --- | --- | --- |
| `learning_rate` | Nova taxa de aprendizado | 1e-5 | Taxa 100x menor que inicial para ajustes sutis sem "quebrar" features úteis já aprendidas |

In [ ]:
model_fine_checkpoint = ModelCheckpoint(
    'best_model_fine.keras',
    monitor='val_auc',
    mode='max',
    save_best_only=True
)

# Descongela algumas camadas para fine-tuning
base_model = model.layers[1]
base_model.trainable = True

# Congela as primeiras N camadas (ajuste conforme necessário)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompila com learning rate menor
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Continua o treinamento
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping, model_fine_checkpoint]
)

try:
    # Combina históricos
    combined_history = {
        k: history.history[k] + history_fine.history[k] 
        for k in history.history
    }

    plot_history(combined_history)
except:
    print(history.history.keys())
    print(history_fine.history.keys())


### 7. Salvamento

In [ ]:
# Apenas para Backup
model.save('cancer_detection_model.keras')

## 8. Testes

In [33]:
# === Função para carregar imagem ===
def load_and_preprocess_image(image_path, image_size=(224, 224)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(image_size)
    img = np.array(img) / 255.0  # Normaliza igual no treino
    img = np.expand_dims(img, axis=0)  # Adiciona batch dimension
    return img

# === Função para testar os modelos ===
def tests():
    results_dict = dict()
    
    models = ('best_model.keras', 'best_model_fine.keras')
    
    for model in models:
        tf_model = tf.keras.models.load_model(model)
        results_dict[model.split('.')[0]] = dict()
        
        for i in range(3000, 3100, 1):
            # === Coleta de Resultados Benignos ===
            img = load_and_preprocess_image(f'../dataset/Benign/{i}.jpg')
            results_dict[model.split('.')[0]][f'Benign/{i}.jpg'] = 1 if tf_model.predict(img)[0][0] < 0.5 else 0
            
            # === Coleta de Resultados Malignos === 
            img = load_and_preprocess_image(f'../dataset/Malignant/{i}.jpg')
            results_dict[model.split('.')[0]][f'Malignant/{i}.jpg'] = 1 if tf_model.predict(img)[0][0] >= 0.5 else 0
            
    return results_dict
            
df = pd.DataFrame(tests())

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [34]:
# === Resultados ===
df.sum()

best_model         175
best_model_fine    193
dtype: int64